### Find Word Group Youtube Link

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

# youtube
sample_num = 10  # 7
time_shift = 0.6

In [4]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
            #df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].tail(sample_num)  # will test
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")\n
    df_search_result is dataframe and "search_string", "start_time", "end_time", "sentence", "video_id" are its columns
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [6]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [7]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [8]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Talk Time 2/Result/2-Find Word Group Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

In [9]:
path_folder_file = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish/Twogram_Threegram_Selected.xlsx"
sheet = "twogram_threegram_selected"  # Sheet1

In [10]:
df_file = pd.read_excel(f"{path_folder_file}", sheet_name=f"{sheet}")
#df_file = pd.read_excel(f"twogram.xlsx")
df_file

,twogram
0,bana da
1,bana mı
2,bana ne
3,ben de
4,benim için
5,bir çok
6,bir daha
7,bir de
8,bir şey
9,bu çok


In [11]:
search_list = df_file.iloc[:,0].to_list()
len(search_list)

31

In [12]:
#disable_video_id_list = ["H6E6N70jYqI","BoaYsdPtJYA","HUZINsU40Fk","Et3diPcEmfY"]
disable_video_id_list = []

In [13]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [14]:
df_youtube_sentence = df_youtube_sentence[~df_youtube_sentence["video_id"].isin(disable_video_id_list)]
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [15]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [16]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU
3,10.267,11.394,mumu,fI1BI4d5KFU
4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [17]:
df_word_group = word_group_youtube(df_youtube_sentence, search_list, "sentence", sample_num)
df_word_group

,search_string,start_time,end_time,sentence,video_id
0,bana da,44.402,47.227,babam abime de bana da iki kardeşiz şunu söyle...,CepFJrGCWzA
1,bana da,211.394,215.394,bana da bol bol şans getir tamam mı çiçeğim se...,jcqVzKEV9kI
2,bana da,364.850,372.620,da devriye yapıyormuş ormancılar zaten dolaşıy...,UIJMtBZOhRg
3,bana da,6696.925,6698.401,bana da silah ver,MvEFmdxPhDo
4,bana da,2074.789,2078.473,ne yapalım sen gider izlersin bana da anlatırs...,LjQR8qT8s_s
...,...,...,...,...,...
305,var mı,1651.788,1653.363,dedenizin bu durumdan haberi var mı acaba,ylA9LrcU9vo
306,var mı,2537.907,2540.329,peki öyle kocandan kaçar gibi gitmek var mı,jluoByJbQgg
307,var mı,2052.030,2064.720,cevap verdi bakkalda vardır ben de toptancısı ...,50wYL3oH1NA
308,var mı,2144.058,2145.009,var mı başka bir şey,r8IFPvdG40s


In [18]:
df_word_group_time_loc = word_group_time_loc(df_word_group, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id
0,bana da,45.177490,45.676020,babam abime de bana da iki kardeşiz şunu söyle...,CepFJrGCWzA
1,bana da,211.394000,211.910129,bana da bol bol şans getir tamam mı çiçeğim se...,jcqVzKEV9kI
2,bana da,369.480606,370.186970,da devriye yapıyormuş ormancılar zaten dolaşıy...,UIJMtBZOhRg
3,bana da,6696.925000,6697.619588,bana da silah ver,MvEFmdxPhDo
4,bana da,2076.767444,2077.381444,ne yapalım sen gider izlersin bana da anlatırs...,LjQR8qT8s_s
...,...,...,...,...,...
305,var mı,1652.863610,1653.170927,dedenizin bu durumdan haberi var mı acaba,ylA9LrcU9vo
306,var mı,2539.934721,2540.329000,peki öyle kocandan kaçar gibi gitmek var mı,jluoByJbQgg
307,var mı,2062.155000,2063.235000,cevap verdi bakkalda vardır ben de toptancısı ...,50wYL3oH1NA
308,var mı,2144.058000,2144.390850,var mı başka bir şey,r8IFPvdG40s


In [19]:
df_word_group_time_loc.start_time = df_word_group_time_loc.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc.end_time = df_word_group_time_loc.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id
0,bana da,44.577490,46.276020,babam abime de bana da iki kardeşiz şunu söyle...,CepFJrGCWzA
1,bana da,210.794000,212.510129,bana da bol bol şans getir tamam mı çiçeğim se...,jcqVzKEV9kI
2,bana da,368.880606,370.786970,da devriye yapıyormuş ormancılar zaten dolaşıy...,UIJMtBZOhRg
3,bana da,6696.325000,6698.219588,bana da silah ver,MvEFmdxPhDo
4,bana da,2076.167444,2077.981444,ne yapalım sen gider izlersin bana da anlatırs...,LjQR8qT8s_s
...,...,...,...,...,...
305,var mı,1652.263610,1653.770927,dedenizin bu durumdan haberi var mı acaba,ylA9LrcU9vo
306,var mı,2539.334721,2540.929000,peki öyle kocandan kaçar gibi gitmek var mı,jluoByJbQgg
307,var mı,2061.555000,2063.835000,cevap verdi bakkalda vardır ben de toptancısı ...,50wYL3oH1NA
308,var mı,2143.458000,2144.990850,var mı başka bir şey,r8IFPvdG40s


In [20]:
df_word_group_time_loc.start_time = df_word_group_time_loc.start_time.apply(lambda x: round(x))
df_word_group_time_loc.end_time = df_word_group_time_loc.end_time.apply(lambda x: round(x))
df_word_group_time_loc 

,search_string,start_time,end_time,sentence,video_id
0,bana da,45,46,babam abime de bana da iki kardeşiz şunu söyle...,CepFJrGCWzA
1,bana da,211,213,bana da bol bol şans getir tamam mı çiçeğim se...,jcqVzKEV9kI
2,bana da,369,371,da devriye yapıyormuş ormancılar zaten dolaşıy...,UIJMtBZOhRg
3,bana da,6696,6698,bana da silah ver,MvEFmdxPhDo
4,bana da,2076,2078,ne yapalım sen gider izlersin bana da anlatırs...,LjQR8qT8s_s
...,...,...,...,...,...
305,var mı,1652,1654,dedenizin bu durumdan haberi var mı acaba,ylA9LrcU9vo
306,var mı,2539,2541,peki öyle kocandan kaçar gibi gitmek var mı,jluoByJbQgg
307,var mı,2062,2064,cevap verdi bakkalda vardır ben de toptancısı ...,50wYL3oH1NA
308,var mı,2143,2145,var mı başka bir şey,r8IFPvdG40s


In [21]:
df_word_group_time_loc["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc['video_id'].map(str)+"&t="+df_word_group_time_loc['start_time'].map(str)+"s"
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id,video_url
0,bana da,45,46,babam abime de bana da iki kardeşiz şunu söyle...,CepFJrGCWzA,https://www.youtube.com/watch?v=CepFJrGCWzA&t=45s
1,bana da,211,213,bana da bol bol şans getir tamam mı çiçeğim se...,jcqVzKEV9kI,https://www.youtube.com/watch?v=jcqVzKEV9kI&t=...
2,bana da,369,371,da devriye yapıyormuş ormancılar zaten dolaşıy...,UIJMtBZOhRg,https://www.youtube.com/watch?v=UIJMtBZOhRg&t=...
3,bana da,6696,6698,bana da silah ver,MvEFmdxPhDo,https://www.youtube.com/watch?v=MvEFmdxPhDo&t=...
4,bana da,2076,2078,ne yapalım sen gider izlersin bana da anlatırs...,LjQR8qT8s_s,https://www.youtube.com/watch?v=LjQR8qT8s_s&t=...
...,...,...,...,...,...,...
305,var mı,1652,1654,dedenizin bu durumdan haberi var mı acaba,ylA9LrcU9vo,https://www.youtube.com/watch?v=ylA9LrcU9vo&t=...
306,var mı,2539,2541,peki öyle kocandan kaçar gibi gitmek var mı,jluoByJbQgg,https://www.youtube.com/watch?v=jluoByJbQgg&t=...
307,var mı,2062,2064,cevap verdi bakkalda vardır ben de toptancısı ...,50wYL3oH1NA,https://www.youtube.com/watch?v=50wYL3oH1NA&t=...
308,var mı,2143,2145,var mı başka bir şey,r8IFPvdG40s,https://www.youtube.com/watch?v=r8IFPvdG40s&t=...


In [22]:
word_count_result(df_word_group_time_loc, ["search_string"], set_condition=True)

,word,word_count
0,ne,6
1,değil,5
2,bu,5
3,için,5
4,bir,4
5,bana,3
6,daha,3
7,de,3
8,da,3
9,kadar,3


In [23]:
df_word_group_time_loc.to_excel(f"{lang_folder.capitalize()}_Word_Group_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx", index=False) 

#### Copy Move And Delete

In [31]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx")
output_file

['Turkish_Word_Group_With_200_Word_10_Youtube_0.6s_Timeshift_For_Talk_Time_Result.xlsx']

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass